In [10]:
#based on "Let's build GPT: from scratch, in code, spelled out." from Andrej Karpathy

# download Quo Vadis from Henryk Sienkiewicz
!wget "http://biblioteka.kijowski.pl/sienkiewicz%20henryk/quo%20vadis%20(ang).txt"
!mv "quo vadis (ang).txt" quo_vadis.txt

--2024-05-12 18:14:04--  http://biblioteka.kijowski.pl/sienkiewicz%20henryk/quo%20vadis%20(ang).txt
Resolving biblioteka.kijowski.pl (biblioteka.kijowski.pl)... 77.65.215.2
Connecting to biblioteka.kijowski.pl (biblioteka.kijowski.pl)|77.65.215.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642072 (627K) [text/plain]
Saving to: ‘quo vadis (ang).txt.1’

quo vadis (ang).txt 100%[===================>] 627,02K  1,14MB/s    in 0,5s    

2024-05-12 18:14:05 (1,14 MB/s) - ‘quo vadis (ang).txt.1’ saved [642072/642072]



In [18]:
#get pointer to read text
with open('quo_vadis.txt', encoding='latin-1') as f:
    text = f.read()

In [22]:
print('len of text: {}'.format(len(text)))
print(text[0:1000])

len of text: 630170
Project Gutenberg EBook, Quo Vadis, by Henryk Sienkiewicz

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**EBooks Readable By Both Humans and By Computers, Since 1971**

*****These EBooks Were Prepared By Thousands of Volunteers*****



Title: Quo Vadis A Narrative of th

In [27]:
#get all unique charaters that occure in text
chars = sorted(list(set(text)))
print(chars)
print("vocabulary length: {}".format(len(chars)))

['\n', ' ', '!', '"', '#', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x8c', '\x9c', 'Æ', 'æ', 'ç', 'é', 'ë', 'ô']
vocabulary length: 86


In [29]:
# transform characters to numeric data
strToInt = {ch:i for i,ch in enumerate(chars)}
intToStr = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[strToInt[c] for c in s] #encode string to integers
decode = lambda l:''.join([intToStr[i] for i in l]) #decode intigers to string

encoded = encode('quo vadis domine')
print('Encoded:', encoded)
print('Decoded:', decode(encoded))

Encoded: [68, 72, 66, 1, 73, 52, 55, 60, 70, 1, 55, 66, 64, 60, 65, 56]
Decoded: quo vadis domine


In [30]:
#encoding entire "quo vadis" text
import torch #PyTorch lib is used for building transformer components https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long) #encode whole text
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([630170]) torch.int64
tensor([39, 69, 66, 61, 56, 54, 71,  1, 30, 72, 71, 56, 65, 53, 56, 69, 58,  1,
        28, 25, 66, 66, 62,  9,  1, 40, 72, 66,  1, 45, 52, 55, 60, 70,  9,  1,
        53, 76,  1, 31, 56, 65, 69, 76, 62,  1, 42, 60, 56, 65, 62, 60, 56, 74,
        60, 54, 77,  0,  0, 26, 66, 67, 76, 69, 60, 58, 59, 71,  1, 63, 52, 74,
        70,  1, 52, 69, 56,  1, 54, 59, 52, 65, 58, 60, 65, 58,  1, 52, 63, 63,
         1, 66, 73, 56, 69,  1, 71, 59, 56,  1, 74, 66, 69, 63, 55, 11,  1, 25,
        56,  1, 70, 72, 69, 56,  1, 71, 66,  1, 54, 59, 56, 54, 62,  1, 71, 59,
        56,  0, 54, 66, 67, 76, 69, 60, 58, 59, 71,  1, 63, 52, 74, 70,  1, 57,
        66, 69,  1, 76, 66, 72, 69,  1, 54, 66, 72, 65, 71, 69, 76,  1, 53, 56,
        57, 66, 69, 56,  1, 55, 66, 74, 65, 63, 66, 52, 55, 60, 65, 58,  1, 66,
        69,  1, 69, 56, 55, 60, 70, 71, 69, 60, 53, 72, 71, 60, 65, 58,  0, 71,
        59, 60, 70,  1, 66, 69,  1, 52, 65, 76,  1, 66, 71, 59, 56, 69,  1, 39,
       

In [32]:
#split data into train and validation sets
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [33]:
block_size = 8
train_data[:block_size + 1]

tensor([39, 69, 66, 61, 56, 54, 71,  1, 30])

In [34]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([39]) the target: 69
when input is tensor([39, 69]) the target: 66
when input is tensor([39, 69, 66]) the target: 61
when input is tensor([39, 69, 66, 61]) the target: 56
when input is tensor([39, 69, 66, 61, 56]) the target: 54
when input is tensor([39, 69, 66, 61, 56, 54]) the target: 71
when input is tensor([39, 69, 66, 61, 56, 54, 71]) the target: 1
when input is tensor([39, 69, 66, 61, 56, 54, 71,  1]) the target: 30


In [35]:
torch.manual_seed(2527)
batch_size = 4 # number of independant sequences processed in parallel
block_size = 8 # maximum context length of predictions

def get_batch(split):
    # generate batch of data of input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[56, 70, 54, 52, 67, 56, 11,  3],
        [73, 56, 69,  9,  1, 59, 60, 70],
        [ 1, 24, 71,  1, 52, 63, 63,  1],
        [60, 70,  0, 53, 66, 55, 76, 11]])
targets:
torch.Size([4, 8])
tensor([[70, 54, 52, 67, 56, 11,  3,  0],
        [56, 69,  9,  1, 59, 60, 70,  1],
        [24, 71,  1, 52, 63, 63,  1, 71],
        [70,  0, 53, 66, 55, 76, 11,  1]])
----
when input is [56] the target: 70
when input is [56, 70] the target: 54
when input is [56, 70, 54] the target: 52
when input is [56, 70, 54, 52] the target: 67
when input is [56, 70, 54, 52, 67] the target: 56
when input is [56, 70, 54, 52, 67, 56] the target: 11
when input is [56, 70, 54, 52, 67, 56, 11] the target: 3
when input is [56, 70, 54, 52, 67, 56, 11, 3] the target: 0
when input is [73] the target: 56
when input is [73, 56] the target: 69
when input is [73, 56, 69] the target: 9
when input is [73, 56, 69, 9] the target: 1
when input is [73, 56, 69, 9, 1] the target: 59
when input is [7